In [2]:
from layers import *


L = ConvolutionalLayer2D(32, 3, 1, 1)

X = np.load("../data/Training_Set/preprocessed_numpy/1.npy")

print("input shape is: ", X.shape)
print(L.forward(X).shape)

input shape is:  (224, 224, 3)
(1, 32, 224, 3)


In [3]:
import numpy as np
import pandas as pd
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

df_train = pd.read_csv("../data/Training_Set/train_labels.csv")
df_test = pd.read_csv("../data/Evaluation_set/eval_labels.csv")

In [4]:
## adding .npy to the end of the file name in ID column
df_train["ID"] = df_train["ID"].apply(lambda x: str(x) + ".npy")
df_test["ID"] = df_test["ID"].apply(lambda x: str(x) + ".npy")

In [5]:
df_train.tail()

,ID,DR,MH,TSLN,ODC,Disease_Risk
944,1909.npy,0,0,1,0,1
945,1910.npy,0,0,1,0,0
946,1914.npy,0,0,1,0,1
947,1915.npy,0,0,1,0,1
948,1917.npy,1,0,0,0,0


In [6]:
df_test.tail()

,ID,DR,MH,TSLN,ODC,Disease_Risk
290,608.npy,1,0,0,0,0
291,611.npy,0,0,1,1,0
292,613.npy,0,0,1,0,0
293,636.npy,0,0,1,0,1
294,637.npy,0,0,1,0,0


In [7]:
# creating an array X with all the files in the df id column

X_train = []
for idx, row in df_train.iterrows():
    array = np.load("../data/Training_Set/preprocessed_numpy/" + row["ID"])
    X_train.append(array)

X_test = []
for idx, row in df_test.iterrows():
    array = np.load("../data/Evaluation_set/preprocessed_eval/" + row["ID"])
    X_test.append(array)

In [8]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [9]:
Y_train = df_train.drop(['ID', 'Disease_Risk'], axis=1)
Y_train = Y_train.to_numpy()

Y_test = df_test.drop(['ID', 'Disease_Risk'], axis=1)
Y_test = Y_test.to_numpy()

In [10]:
print("shape of X_train is: ", X_train.shape)
print("shape of Y_train is: ", Y_train.shape)
print("shape of X_test is: ", X_test.shape)
print("shape of Y_test is: ", Y_test.shape)

shape of X_train is:  (949, 224, 224, 3)
shape of Y_train is:  (949, 4)
shape of X_test is:  (295, 224, 224, 3)
shape of Y_test is:  (295, 4)


In [13]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

datagen.fit(X_train)

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

## creating a model for multi-label classification
model = Sequential()
model.add(Conv2D(512, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Flatten())
model.add(Dense(4, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

model.summary()

model.fit(datagen.flow(X_train, Y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32, epochs=10)

model.evaluate(X_test, Y_test)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 222, 222, 512)     14336     
                                                                 
 conv2d_13 (Conv2D)          (None, 220, 220, 512)     2359808   
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 110, 110, 512)    0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 108, 108, 256)     1179904   
                                                                 
 conv2d_15 (Conv2D)          (None, 106, 106, 256)     590080    
                                                                 
 conv2d_16 (Conv2D)          (None, 104, 104, 256)     590080    
                                                      

[0.7835941910743713, 0.5457627177238464]